This notebook aims to train and validate the softmax classifier as to compare its performance to that of an LSTM neural network for hate speech classification

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

from tensorflow import one_hot
from tensorflow.keras.preprocessing.text import Tokenizer

from utilities.data_preprocessors import read_preprocess, series_to_1D_array, construct_embedding_dict, construct_embedding_matrix, sentence_to_avg
from utilities.data_visualizers import train_cross_results_v2, view_final_metrics
from models.model_arcs import load_lstm_model

%load_ext autoreload
%autoreload 2

# Load data

In [ ]:
# 1 for religious and 0 for non religious
df = pd.read_csv('./data/hate-speech-data-cleaned.csv', index_col=0)
df = read_preprocess(df)

all_words = pd.Series(series_to_1D_array(df['comment']))
all_unique_words_counts = all_words.value_counts()
all_unique_words = all_words.unique()


In [ ]:
df['label'].value_counts()

# Preprocess data

In [ ]:
# rejoin the comment columns values of lists of words to sentences
df['comment'] = df['comment'].apply(lambda comment: " ".join(comment))
df

In [ ]:
# at the same time one hot encode the y labels/classes
len_unique_labels = len(df['label'].unique())
Y_oh = one_hot(df['label'], len_unique_labels).numpy()
Y_oh

# Retrieving and assigning important variables for training classifier

In [ ]:
sents = df['comment']

# get number of all unique words
num_words_3 = len(all_unique_words)

# instantiate Tokenizer on the total number of all unique words
tokenizer = Tokenizer(num_words=num_words_3, split=' ')

# call .fit_on_texts to create the word_index and index_word dicts
tokenizer.fit_on_texts(sents)

# save the tokenizer dictionaries for use later when loading GloVe embeddings
word_to_index = tokenizer.word_index
index_to_word = tokenizer.index_word
print(len(word_to_index))
print(word_to_index)

In [ ]:
# important variables

# includes oov words
emb_dict, emb_vec_len = construct_embedding_dict('./embeddings/glove.42B.300d.txt', word_to_index)
emb_matrix = construct_embedding_matrix(word_to_index, emb_dict, emb_vec_len)

# Transform all sentences to word vectors

In [ ]:
vect_sents = sents.apply(lambda sentence: sentence_to_avg(sentence, emb_dict))

In [ ]:

# # predict probabilities for test set
# yhat_probs = model.predict(testX, verbose=0)
# # predict crisp classes for test set
# yhat_classes = model.predict_classes(testX, verbose=0)

# # reduce to 1d array
# yhat_probs = yhat_probs[:, 0]
# yhat_classes = yhat_classes[:, 0]

# # accuracy: (tp + tn) / (p + n)
# accuracy = accuracy_score(testy, yhat_classes)
# print('Accuracy: %f' % accuracy)
# # precision tp / (tp + fp)
# precision = precision_score(testy, yhat_classes)
# print('Precision: %f' % precision)
# # recall: tp / (tp + fn)
# recall = recall_score(testy, yhat_classes)
# print('Recall: %f' % recall)
# # f1: 2 tp / (2 tp + fp + fn)
# f1 = f1_score(testy, yhat_classes)
# print('F1 score: %f' % f1)

# matrix = confusion_matrix(testy, yhat_classes)
# print(matrix)